In [1]:
table={
    "LDI": '000',
    "ST": '001',
    "ADD": '010',
    "SUB": '011',
    "ADDI": '100',
    "LD": '101'
}

array_table={
    "a0": '000',
    "a1": '001',
    "a2": '010',
    "a3": '011',
    "a4": '100'
}

reg_table={
    "r0": '000',
    "r1": '001',
    "r2": '010',
    "r3": '011',
    "r4": '100'
}

In [2]:
def convert(instruction):
    writef=open('bin.txt', 'a')
    instr=instruction.split()
    print(instr)
    binins=[table[instr[0]]]
    if instr[0] == 'LDI':
      binins.append(array_table[instr[1]])
      buff=bin(int(instr[2]))[2:]
      if len(buff)<3:
        while len(buff)<3:
          buff='0'+buff
      binins.append(buff)
      for i in range(int(instr[2])):
        buff=bin(int(instr[i+3]))[2:]
        if len(buff)<3:
          while len(buff)<3:
            buff='0'+buff
        binins.append(buff)
    if instr[0] == 'LD':
      binins.append(array_table[instr[1]])
      binins.append(array_table[instr[2]])
    if instr[0]=='ADD' or instr[0]=='SUB':
      binins.append(array_table[instr[1]])
      binins.append(array_table[instr[2]])
      binins.append(array_table[instr[3]])
    if instr[0]=='ADDI':
      binins.append(array_table[instr[1]])
      binins.append(array_table[instr[2]])
      buff=bin(int(instr[3]))[2:]
      if len(buff)<3:
        while len(buff)<3:
          buff='0'+buff
      binins.append(buff)
    if instr[0]=='ST':
      buff=bin(int(instr[1][0]))[2:]
      if len(buff)<3:
        while len(buff)<3:
          buff='0'+buff
      binins.append(buff)
      binins.append(array_table[instr[1][2:-1]])
      binins.append(reg_table[instr[2]])

    binstr=''
    for i in binins:
      binstr=binstr+str(i)
    print(binstr)
    writef.write(binstr)
    writef.write('\n')

    # print(binins)


f = open('simd_instructions.txt', 'r')
for i in f:
  convert(i)


['LDI', 'a0', '3', '1', '2', '3']
000000011001010011
['LDI', 'a2', '3', '4', '5', '6']
000010011100101110
['LD', 'a1', 'a2']
101001010
['ADD', 'a2', 'a0', 'a1']
010010000001
['SUB', 'a3', 'a0', 'a1']
011011000001
['ADDI', 'a4', 'a0', '2']
100100000010
['ST', '0(a2)', 'r0']
001000010000
['ST', '4(a2)', 'r1']
001100010001
['ST', '0(a3)', 'r2']
001000011010
['ST', '4(a4)', 'r3']
001100100011


In [3]:
arr_sim={
    '000':[],
    '001':[],
    '010':[],
    '011':[],
    '100':[],
}

reg_sim={
    '000':0,
    '001':0,
    '010':0,
    '011':0,
    '100':0,
}

In [4]:
f=open('bin.txt', 'r')
arrays=[]
for istr in f:
  iarr=[istr[i:i+3] for i in range(0, len(istr), 3)]
  iarr.pop()
  print(iarr)
  if iarr[0]=='000':
    n=int(iarr[2], 2)
    buff=[]
    for i in range(n):
      buff.append(int(iarr[3+i], 2))
    arr_sim[iarr[1]]=buff
  if iarr[0] == '101':
    arr_sim[iarr[1]]=arr_sim[iarr[2]]
  if iarr[0] == '010':
    for i in range(len(arr_sim[iarr[2]])):
      arr_sim[iarr[1]].append(arr_sim[iarr[2]][i]+arr_sim[iarr[3]][i])
  if iarr[0] == '011':
    for i in range(len(arr_sim[iarr[2]])):
      arr_sim[iarr[1]].append(arr_sim[iarr[2]][i]-arr_sim[iarr[3]][i])
  if iarr[0]== '100':
    x=int(iarr[3], 2)
    for i in range(len(arr_sim[iarr[2]])):
      arr_sim[iarr[1]].append(arr_sim[iarr[2]][i]+x)
  if iarr[0]=='001':
    # print(int(iarr[1], 2))
    index=int(iarr[1], 2)//4
    reg_sim[iarr[3]]=arr_sim[iarr[2]][index]



['000', '000', '011', '001', '010', '011']
['000', '010', '011', '100', '101', '110']
['101', '001', '010']
['010', '010', '000', '001']
['011', '011', '000', '001']
['100', '100', '000', '010']
['001', '000', '010', '000']
['001', '100', '010', '001']
['001', '000', '011', '010']
['001', '100', '100', '011']


In [5]:
print(arr_sim)
print(reg_sim)

{'000': [1, 2, 3], '001': [4, 5, 6, 5, 7, 9], '010': [4, 5, 6, 5, 7, 9], '011': [-3, -3, -3], '100': [3, 4, 5]}
{'000': 4, '001': 5, '010': -3, '011': 4, '100': 0}


In [6]:
pipeline = {"F": "", "D": "", "X": "", "M": "", "W": ""}

In [7]:
cyc=2
reg=[]
dep_reg=[]

def fit(ins):
  global reg
  if ins[0]== "001":
    reg.append(ins[3])
    dep_reg.append([ins[2]])
  if ins[0]=='000':
    reg.append(ins[1])
    dep_reg.append(-1)
  if ins[0]=='010' or ins[0]=='011':
    reg.append(ins[1])
    buff=[ins[2], ins[3]]
    dep_reg.append(buff)
  if ins[0]=='100' or ins[0]=='101':
    reg.append(ins[1])
    dep_reg.append([ins[2]])

b = open('bin.txt', 'r')
for istr in b:
  x=[istr[i:i+3] for i in range(0, len(istr), 3)]
  fit(x)

In [8]:
for i in range(len(reg)):
  print(reg[i], " ", dep_reg[i])

000   -1
010   -1
001   ['010']
010   ['000', '001']
011   ['000', '001']
100   ['000']
000   ['010']
001   ['010']
010   ['011']
011   ['100']


In [9]:
stalls=0
x=-1
for i in dep_reg:
  x+=1
  if i!= -1:
    mr=[]
    for dreg in i:
      for tarreg in reg:
        if dreg==tarreg:
          if reg.index(tarreg)-x in range(-2, 0):
            if type(dep_reg[x-1]) != int and dreg in dep_reg[x-1]:
              break
            mr.append(3-abs(reg.index(tarreg)-x))
    if mr:
      stalls+=max(mr)

cycles=len(reg)+4+stalls
print(cycles)

18
